In [5]:
from collections import Counter
import numpy as np

def rouge_n(evaluated_sentences, reference_sentences, n=1):
    evaluated_ngrams = [tuple(evaluated_sentences[i:i+n]) for i in range(len(evaluated_sentences)-n+1)]
    reference_ngrams = [tuple(reference_sentences[i:i+n]) for i in range(len(reference_sentences)-n+1)]
    
    evaluated_ngrams_count = Counter(evaluated_ngrams)
    reference_ngrams_count = Counter(reference_ngrams)
    
    overlapping_ngrams_count = sum((evaluated_ngrams_count & reference_ngrams_count).values())
    reference_ngrams_total = sum(reference_ngrams_count.values())
    
    if reference_ngrams_total == 0:
        recall = 0
    else:
        recall = overlapping_ngrams_count / reference_ngrams_total
    
    return recall

def rouge_l(evaluated_sentences, reference_sentences):
    lcs = np.zeros((len(evaluated_sentences)+1, len(reference_sentences)+1))
    
    for i in range(1, len(evaluated_sentences)+1):
        for j in range(1, len(reference_sentences)+1):
            if evaluated_sentences[i-1] == reference_sentences[j-1]:
                lcs[i][j] = lcs[i-1][j-1] + 1
            else:
                lcs[i][j] = max(lcs[i-1][j], lcs[i][j-1])
    
    return lcs[len(evaluated_sentences)][len(reference_sentences)] / max(len(evaluated_sentences), len(reference_sentences))

def rouge(evaluated_sentences, reference_sentences):
    recall = rouge_n(evaluated_sentences, reference_sentences) # ROUGE-N
    precision = rouge_n(reference_sentences, evaluated_sentences) # ROUGE-N
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    rouge_l_score = rouge_l(evaluated_sentences, reference_sentences) # ROUGE-L
    return {'ROUGE-N Recall': recall, 'ROUGE-N Precision': precision, 'ROUGE-N F1': f1_score, 'ROUGE-L': rouge_l_score}


In [6]:
# Example usage:
reference_summary = r"Let any one now consider with what little difficulty the king could have maintained his position in Italy had he observed the rules above laid down, and kept all his friends secure and protected; for although they were numerous they were both weak and timid, some afraid of the Church, some of the Venetians, and thus they would always have been forced to stand in with him, and by their means he could easily have made himself secure against those who remained powerful. But he was no sooner in Milan than he did the contrary by assisting Pope Alexander to occupy the Romagna. It never occurred to him that by this action he was weakening himself, depriving himself of friends and of those who had thrown themselves into his lap, whilst he aggrandized the Church by adding much temporal power to the spiritual, thus giving it greater authority. And having committed this prime error, he was obliged to follow it up, so much so that, to put an end to the ambition of Alexander, and to prevent his becoming the master of Tuscany, he was himself forced to come into Italy. And as if it were not enough to have aggrandized the Church, and deprived himself of friends, he, wishing to have the kingdom of Naples, divided it with the King of Spain, and where he was the prime arbiter in Italy he takes an associate, so that the ambitious of that country and the malcontents of his own should have somewhere to shelter; and whereas he could have left in the kingdom his own pensioner as king, he drove him out, to put one there who was able to drive him, Louis, out in turn. The wish to acquire is in truth very natural and common, and men always do so when they can, and for this they will be praised not blamed; but when they cannot do so, yet wish to do so by any means, then there is folly and blame. Therefore, if France could have attacked Naples with her own forces she ought to have done so; if she could not, then she ought not to have divided it. And if the partition which she made with the Venetians in Lombardy was justified by the excuse that by it she got a foothold in Italy, this other partition merited blame, for it had not the excuse of that necessity. Therefore Louis made these five errors: he destroyed the minor powers, he increased the strength of one of the greater powers in Italy, he brought in a foreign power, he did not settle in the country, he did not send colonies. Which errors, had he lived, were not enough to injure him had he not made a sixth by taking away their dominions from the Venetians; because, had he not aggrandized the Church, nor brought Spain into Italy, it would have been very reasonable and necessary to humble them; but having first taken these steps, he ought never to have consented to their ruin, for they, being powerful, would always have kept off others from designs on Lombardy, to which the Venetians would never have consented except to become masters themselves there; also because the others would not wish to take Lombardy from France in order to give it to the Venetians, and to run counter to both they would not have had the courage. And if any one should say: 'King Louis yielded the Romagna to Alexander and the kingdom to Spain to avoid war,' I answer for the reasons given above that a blunder ought never to be perpetrated to avoid war, because it is not to be avoided, but is only deferred to your disadvantage. And if another should allege the pledge which the king had given to the Pope that he would assist him in the enterprise, in exchange for the dissolution of his marriage(*) and for the cap to Rouen,(+) to that I reply what I shall write later on concerning the faith of princes, and how it ought to be kept."
generated_summary = r"On the subject of colonies, Machiavelli goes one step further, noting that it is better to displace or disrupt the poor and powerless than the rich and powerful.', 'Granted this opportunity, Louis proceeded to squander it.', 'He put down the weaker powers , increased the strength of a major power , introduced a powerful foreigner into the fray , never took up residence in Italy, never set up colonies, and deprived the Venetians of their power.', 'This last error proved fatal: if the Venetians had retained full power, no one would have taken Lombardy from France just to give it to Venice, and the Venetians would not have let others in."
reference_sentences = reference_summary.split()
generated_sentences = generated_summary.split()

scores = rouge(generated_sentences, reference_sentences)
print(scores)

{'ROUGE-N Recall': 0.08823529411764706, 'ROUGE-N Precision': 0.5405405405405406, 'ROUGE-N F1': 0.15170670037926676, 'ROUGE-L': 0.05588235294117647}
